In [ ]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import os

In [ ]:
def augment_brightness_camera_images(image):

    image1 = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    random_bright = .25 + np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1, cv2.COLOR_HSV2RGB)
    return image1

In [ ]:
def transform_image(img, ang_range, shear_range, trans_range, brightness=False):
    '''
    This function transforms images to generate new images.
    The function takes in following arguments,
    1- Image
    2- ang_range: Range of angles for rotation
    3- shear_range: Range of values to apply affine transform to
    4- trans_range: Range of values to apply translations over.

    A Random uniform distribution is used to generate different parameters for transformation

    '''
    # Rotation
    ang_rot = np.random.uniform(ang_range) - ang_range/2
    rows,cols,ch = img.shape
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2), ang_rot, 1)

    # Translation
    tr_x = trans_range*np.random.uniform() - trans_range/2
    tr_y = trans_range*np.random.uniform() - trans_range/2
    Trans_M = np.float32([[1,0,tr_x], [0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5], [20,5], [5,20]])

    pt1 = 5 + shear_range*np.random.uniform() - shear_range/2
    pt2 = 20 + shear_range*np.random.uniform() - shear_range/2

    # Brightness
    pts2 = np.float32([[pt1,5], [pt2,pt1], [5,pt2]])

    shear_M = cv2.getAffineTransform(pts1, pts2)

    img = cv2.warpAffine(img, Rot_M, (cols,rows))
    img = cv2.warpAffine(img, Trans_M, (cols,rows))
    img = cv2.warpAffine(img, shear_M, (cols,rows))

    if brightness:
      img = augment_brightness_camera_images(img)

    return img

In [ ]:
PATH = '/data3/marcolini/myq_dataset/Selfie-dataset/labelled/'
AUG_PATH = '/data3/marcolini/myq_dataset/Selfie-dataset/augmented/'

if os.path.exists(PATH + '/.DS_Store'):
    os.remove(PATH + '/.DS_Store')

l_images = os.listdir(PATH)

In [ ]:
l_images

In [ ]:
filenames = [] 
for i in l_images:
    isplit = i.split('_')
    cl_sex = isplit[1]
    cl_mood = isplit[0]
    if cl_sex == '0' and cl_mood == '0': # select only males with negative emotions
        filenames.append(i)

In [ ]:
for f in filenames:
    for i in range(2):
        image = cv2.imread(os.path.join(PATH, f))
        image = transform_image(image, 20, 10, 1, brightness=True)

        new_f = f.split('.')[0] + '_' + str(i) + '.' + f.split('.')[1]
        cv2.imwrite(os.path.join(AUG_PATH, new_f) , image)
        